In [ ]:
#!pip install transformers torch pytesseract
#! pip install sentencepiece sacremoses

from transformers import pipeline
import pandas as pd
import os

#pip install google-cloud-storage
from google.colab import auth
auth.authenticate_user()


In [ ]:
from google.cloud import storage

# Correct bucket name and file path
bucket_name = "springfield_40k"  # Ensure this matches your actual bucket name
css_file_path = "springfield_10_scripts.csv"  # Use only the relative path, not the full URL

# Initialize GCS client
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(css_file_path)

# Read the CSS content (or CSV in this case)
df = pd.read_csv(blob.open('r'))
df


In [ ]:
df['Script'][3]

Sentiment Analysis

In [ ]:
sentiment_pipe = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment')
grown_sentiment = sentiment_pipe('''
Two-handed.
- Why, because I'm black?
Yes, that's what I thought
Ten seconds.
We're going to win.
There.
Inside.
Just do it, Lenny.
His foot was on the line.
First I want to thank the Earnshaw family
for their summer ...
and to celebrate our victory.
''')
print(grown_sentiment)
#label_2 is good
#label_1 is neutral
#label_0 is bad

Emotion Detection

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

emotion = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

In [ ]:
george = df['Script'][3]

In [ ]:
def chunk_classifier(script):
  humor_list = []
  for i in np.arange(0,len(script), 500):
    print(script)
    #grown_ups = grown_ups.replace('\n',' ')

def split_script(script, max_chars=600):
    chunks = []
    start = 0

    while start < len(script):
        # Find the nearest newline after 400 characters
        end = start + max_chars
        if end < len(script):
            newline_pos = script.rfind("\n", start, end)
            if newline_pos != -1:
                end = newline_pos + 1  # Include the newline
        chunks.append(script[start:end].strip().replace('\n',' '))  # Add chunk, remove leading/trailing spaces
        start = end  # Move to the next chunk

    return chunks

def emotions(script):
  emotions = []
  chunks = split_script(script)
  for chunk in chunks:
    emot = emotion(chunk)[0]['label']
    emotions.append(emot)
  return emotions

def sentiments(script):
  sentiments = []
  chunks = split_script(script)
  for chunk in chunks:
    sentiment = sentiment_pipe(chunk)[0]['label']
    if sentiment[-1] == '0':
      sentiment = -1
    elif sentiment[-1] == '1':
      sentiment = 0
    else: sentiment = 1
    sentiments.append(sentiment)
  return sentiments

  #do a mapping instead - faster


In [ ]:
george_emotion = emotions(george)
george_sentiment = sentiments(george)
#0,2,1,1 original
#should be -1, 1, 0, 0

In [ ]:
import numpy as np
import statistics
#sentiment analysis
np.std(george_sentiment)
#rate of change

#sentiment entropy
count_pos = george_sentiment.count(1)
count_neg = george_sentiment.count(-1)
count3_neut = george_sentiment.count(0)
#print(count_pos, count_neg, count3_neut)
prob_pos, prob_neg, prob_neut= count_pos/len(george_sentiment), count_neg/len(george_sentiment), count3_neut/len(george_sentiment)

sentiment_entropy = -(prob_pos * np.log2(prob_pos) + prob_neg * np.log2(prob_neg) + prob_neut * np.log2(prob_neut))


In [ ]:
#emotion

statistics.mode(george_emotion)

In [ ]:
from collections import Counter

def emotion_frequencies(script):
    emotions = []
    chunks = split_script(script)
    for chunk in chunks:
      emot = emotion(chunk)[0]['label']
      emotions.append(emot)

    # Count occurrences
    counts = Counter(emotions)

    # Total number of occurrences (for normalization)
    total_count = sum(counts.values())

    # Normalize frequencies
    normalized_frequencies = {emotion: freq / total_count for emotion, freq in counts.items()}

    return normalized_frequencies
# Get ranking dictionary

emotion_frequencies(grown_ups)

# Metrics on Sentiment and Emotion

In [ ]:
#Sentiment Entropy
import numpy as np
import statistics
#sentiment analysis
np.std(george_sentiment)
#rate of change

#sentiment entropy
count_pos = george_sentiment.count(1)
count_neg = george_sentiment.count(-1)
count3_neut = george_sentiment.count(0)
#print(count_pos, count_neg, count3_neut)
prob_pos, prob_neg, prob_neut= count_pos/len(george_sentiment), count_neg/len(george_sentiment), count3_neut/len(george_sentiment)

sentiment_entropy = -(prob_pos * np.log2(prob_pos) + prob_neg * np.log2(prob_neg) + prob_neut * np.log2(prob_neut))
sentiment_entropy

In [ ]:
#Sentiment Differentials / Volatility

#1 Calculate differentials array
george_sentiment
diff = np.array(george_sentiment[1:]) - np.array(george_sentiment[:-1])
#1 Caculate differentials standard deviation
np.std(diff), diff

In [ ]:
def sentiment_params(script):
    sentiments = []
    chunks = split_script(script)

    # Define mapping dictionary
    sentiment_mapping = {
        '0': -1,  # Negative
        '1': 0,   # Neutral
        '2': 1    # Positive
    }

    for chunk in chunks:
        sentiment_label = sentiment_pipe(chunk)[0]['label']
        sentiment = sentiment_mapping.get(sentiment_label[-1], 0)  # Default to 0 if not found
        sentiments.append(sentiment)

    #sentiments ready

    #2 Entropy
    count_pos = sentiments.count(1)
    count_neg = sentiments.count(-1)
    count3_neut = sentiments.count(0)
    #print(count_pos, count_neg, count3_neut)
    prob_pos, prob_neg, prob_neut= count_pos/len(sentiments), count_neg/len(sentiments), count3_neut/len(sentiments)

    sentiment_entropy = -(prob_pos * np.log2(prob_pos) + prob_neg * np.log2(prob_neg) + prob_neut * np.log2(prob_neut))

    #3 Std of sentiment
    sentiment_diff = np.array(sentiments[1:]) - np.array(sentiments[:-1])
    sentiment_std = np.std(sentiment_diff)

    return {'sentiment_entropy': sentiment_entropy, 'sentiment_std': sentiment_std}

sentiment_params(grown_ups)

In [ ]:
#Emotion Mapping
def emotion_frequencies(script):
    emotions = []
    chunks = split_script(script)
    for chunk in chunks:
      emot = emotion(chunk)[0]['label']
      emotions.append(emot)

    # Count occurrences
    counts = Counter(emotions)

    # Total number of occurrences (for normalization)
    total_count = sum(counts.values())

    # Normalize frequencies
    normalized_frequencies = {emotion: freq / total_count for emotion, freq in counts.items()}

    return normalized_frequencies
# Get ranking dictionary

emotion_frequencies(grown_ups)

In [ ]:
# Apply function to each row and expand results into new columns
def embedder(df):
    #emotions
    df_emotions = df["Script"].apply(emotion_frequencies).apply(pd.Series)
    #sentiments
    df_sentiments = df["Script"].apply(sentiment_params).apply(pd.Series)
    # Merge new features into original DataFrame
    df_embedded = pd.concat([df, 
                             df_emotions, 
                             df_sentiments],
                            axis=1)

    # Show result
    return df_embedded
embedder(df)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

humor_pipe = pipeline("text-classification", model="mohameddhiab/humor-no-humor")

In [ ]:
humor_pipe(gu_chunks)

In [ ]:
gu_emotion = emotions(grown_ups)
gu_sentiment = sentiments(grown_ups)
#0,2,1,1 original

In [ ]:
gu_emotion

In [ ]:
diff2 = np.array(gu_sentiment[1:]) - np.array(gu_sentiment[:-1])
#1 Caculate differentials standard deviation
np.std(diff2), diff2

In [ ]:
gu_chunks = split_script(grown_ups, max_chars=200)
gu_chunks[:5]

In [ ]:
grown_ups = '''
Two-handed.
- Why, because I'm black?
Yes, that's what I thought
Ten seconds.
We're going to win.
There.
Inside.
Just do it, Lenny.
His foot was on the line.
First I want to thank the Earnshaw family
for their summer ...
and to celebrate our victory.
- The five Ferdinando.
They are dynamite.
- You're Sat, Dad.
We see here the subject of
the children to celebrate, not yours.
I want to talk about what your children
have performed this afternoon.
You played the game like
I always asked.
You gave everything you had.
When the final buzzer went off ...
we were champions.
Promise me something ...
play your life like
the match today.
So as the final buzzer
sounds of your life ...
you have no regrets.
- On the coach.
My first champions.
Stay together.
- Time to die.
I am the biggest agent in Hollywood,
and you just made me angry.
I called Brad Pitt movie for you.
Then I called Julia Roberts.
I do not care.
Yes, you dim indeed better.
Thanks, a nice July 4th.
I love you.
How you doing, guys.
Greg, Keithie?
Once the game
Slides and Ladders heard?
Best fat game.
I played it as a child.
They give you a turner.
You turn and land on a ladder ...
then climb up the ladder.
If you land on a slide ...
then you slide down.
And start again.
What then? The winner gets
a training bra?
Yes, Dad, that sounds bad.
This was the best game I
and my buddies played ...
to show a winter in the basement,
with a hot chocolate.
That would taste best now.
Rita!
Its message. She washes.
Dad, we went yesterday to Spielberg
shop. They have a 150 inch TV.
Are we so alone?
- If you E.T. directs.
Beg for mercy.
- What? You're on 's cruise ship ...
and cut your heads off with a
chainsaw?
Or you can drown them.
- I see it now.
Grandmother overboard.
Plus 200 points.
Can someone answer the phone.
Maybe some chamomile for my throat.
- Message is not the nanny.
How bad is that for a child.
Walk through the corridor itself.
You notice everyone.
- This is work related.
So I can pay for your posts.
I'm all for you guys.
- Who rang?
Sorry, no idea
Becky replied.
Thanks for the drinks.
- Rita, will you poison?
Go back messages.
I could not hear.
What is it on?
I did not mean that.
It was an accident.
It is good.
What did you do now?
I wanted to use the Navi.
What is that?
Navi in my car
a question.
The Navigation system you mean.
- Quiet.
A man called and she said you
Buzzer friend in heaven ...
and I wanted to find heaven in the Navi,
so you could visit.
Gosh.
Who is your friend, Dad?
Buzzer coach was someone special
Dad in his life.
Terrible.
Go drink some hot chocolate.
Sure you're all right?
Eric, honey, your mother.
Eric, are you okay?
Look at that crap.
What is happening?
Hi, Mom.
- Hello.
I see you there.
Is something wrong?
- I tell you later.
I cooked risotto with what you saw
Rachel Ray's summer special.
Terrible, Mom.
- It tastes like 'n run over animals.
Hear how they talk to me?
Say something.
You know that your father is working hard
this tidy house ...
and three times a day to cook food.
- Hello.
By the way ...
I brought along a pizza.
Eat your pizza it.
You broke my heart.
Apparently it's back in the time of the month.
That's bad.
Who regulates funeral?
His niece must be devastated.
Is she still hot?
I lose my temper, Marcus.
- The Juice
What?
If I am not at the funeral, say the
From the police that Arlene was Applebees.
Thanks for calling.
I see you there.
Bye.
The Coach is dead.
Friday is his funeral.
I can not do alone.
Will you come with me?
Of course, I put my catheter operation.
You're my special lady.
Now I bring you to show a world of
relaxation.
NEW ENGLAND
Guys, behave normally.
- What's wrong with 'n sunglasses?
I do not want anyone appeals to me as ...
- Hollywood!
Look, Kurt McKenzie.
The entire McKenzie clan.
My wife is over there.
Go say hello.
That ass is still growing.
There's one in Cardesi.
You turned into a Michael Vick thing.
Hi, Roxanne, do you remember me?
I was at your wedding and danced
constantly on the table. Deanne McKenzie.
Of course.
Good to see you.
You radiate so. You look so
beautiful with your pregnancy.
What are you talking about?
I smile there with me.
You had to see your face ...
Little Lenny Feder.
- I'm not the only grown.
Look at this piece.
And so much success.
Give mommy a kiss.
Where is the cheek.
Gotcha.
Almost in my eye, but it is.
Is your son a houseman now?
My daughter broke her work
to pay taxes ...
stupid home while cleaning.
So that you can talk about me,
when you look as ...
Idi Amin with a propeller on your head.
That's sweet.
- This is my mother.
Hello, Mama ronzoni,
pleasant.
I'm Little Lenny's wife.
What a nice piece you got there, Lenny.
They taste good.
That I would think.
Did you turn off your phone?
Yes, but I have massage therapy.
- What? No cell phones.
If not, then I'm gone.
- Dickie.
Who is ready for the funeral.
Sorry, death makes me weird.
How you doing, McKenzie?
You have soft hands.
You still use Palmolive?
How are you, Lenny.
You would go to the gym?
What do you mean?
- You're fat.
No. I wish you always
proposals to the Olsen twins.
We keep our heads down.
Those are your children?
And that girl? Still no nanny?
No, a friend from China.
I'm on that web site.
- No, an exchange student.
My wife brought her inside.
- Your wife wants girls.
That's nice.
He has always lucky.
I always thought something like that.
- Hey, there.
Lamonsoff.
- A convertible?
So it goes.
- You're bigger than me now.
Are you a B-cup now?
Nice ears, you get the packet there on NLF?
Beautiful cabrio. Is that
a handmade lawn furniture?
I am the co-owner of
a lawn furniture.
I'm the boss now.
- Good for you.
You brought your mother this time.
- A good one.
Seems to shake hands with n hedgehog.
God, it's Captain Caring
the SS melodrama.
This is great to play it.
Terrible conditions.
It sucks.
Good to see you.
He now coaches the team in heaven.
Right.
- He has Willt ...
Justin P, Dr. J.
- It is still alive.
All those types.
You look like Elvis when he
Lumpa was in UMPA.
He is just for me.
Jokes at funerals are
disrespectful, but ...
as that part of your healing,
I think the best.
Nice, thanks.
Find where the best thing is?
Roxanne, you remember
Rob which I had?
Of course, the boy until
his 19th went into puberty, right?
That was our secret.
And this should be your mother.
My wife.
- I'm sorry.
I'm Gloria.
It is so pleasant.
You design your line is so beautiful.
- Thank you.
I love your scarves.
- Thank you.
They tie them down to me later.
He believes it seriously.
So dirty.
These are long tongues.
See you inside.
There?
Put down here, baby.
I'm going to Lenny.
Lenny, how are you?
I've heard ...
of a bird, you guys summerhouse
have rented for the weekend.
A dream come true.
Yes, for a whole weekend.
But we need to Milan tomorrow.
For Roxanne's fashion show.
- That is so sad.
But you are so great in Milan.
J 'Lo and Mark Antony went on vacation there.
Just across the street from the
como lake where George Clooney has a villa.
I read too many magazines.
Can I talk to you.
- Sorry, I'm going away.
He was ashamed.
Tom Cruise was married there too.
Thanks for the notice.
- What?
Why for a whole weekend?
It's vacation, they rent it to you
not for one night.
So your friends partying all weekend,
while you sit in Milan.
That was sweet of you.
- Our dear ...
it was your money.
- It's your, and my money.
All our money together and split it.
Nice hat, brings death
good in the room.
Bit nervous by the funeral.
There he is Hollywood.
- Mr. Greatness.
Do you think he represents Hulk Hogan?
No chance.
And so great is not it, eh.
Anyway, Rita's an exchange student.
I was on your calculator.
Now I know what happened to the fish tank.
Coach Robert "Buzzer" Ferdinando,
was a servant.
A beautiful word to describe
how he had for his fellow man.
He really is.
When he knew his time was almost over ...
He told me that one specific man,
who was talking about him.
Therefore I ask Lenny Feder forward.
Sorry, I'm totally unprepared.
I am extremely honored to
Coach asked me this.
I also feel bad because
I had not seen him for a long time ...
and he always wanted that I passed ...
and I was too busy ...
and I regret it.
I understand that the team from 1978 to
only championship team he once was.
And that is very impressive.
Judging from the talent we
had to offer.
Believe it or not, Rob and Mark
were then even smaller.
They had the talent that you never had.
The ball pass.
Someone must clearly learn
chicken legs to pass.
Just kidding.
I say goodbye to a great
part of our lives.
You have everyone in the room changed.
Hopefully we live our lives just like you ...
and we have no regrets as
the final buzzer sounds.
Let us bow in silence.
This had to stop after you.
This is madness.
Five dollars that he goes on one knee.
- Good.
Grow up.
- Look at him now.
Is not that angry, Rob.
What we find in Rob's girl?
Happy?
Saint Pauli grandma?
- She seems okay.
She's pretty.
- Me with a wig.
His daughter is what.
When Ben Franklin flew that kite
with key on it.
That was at her apartment.
He seems to love.
He was also in love with the first three women.
The impostor, the beater and the eater.
He was always madly in love so suddenly.
And one day he snaps,
and was quickly over.
Coffee, Mr. Feder?
No thanks.
Voluntarily offer coffee.
Drink it and learn, but what in the church.
But you reported me you wanted coffee.
- No, you drunk and studied.
So you study books in the church.
Come on, Rita.
Mama, I want some milk.
- Sure, come.
I'll give you something.
Quiet, not a big gulps.
You get stomach ache.
Your son is so cute.
How old is he?
He is 48 months.
That's four.
We wanted to quit last year,
but he so keenly desire ...
and we do not like to say no.
Honey, you can use a fork?
No!
She apparently got the memo.
I want that spot.
Respect the convertibles.
Nice that you took your piggy bank.
Look at you again.
- Come on.
Rob his car is also an UMPA Lumpa.
It's cute.
It is enough for one night.
Can you handle it?
Daddy, what does that machine?
- It takes away the moths.
Where?
- Hell.
Do not say that.
- Sorry, not hell, Mexico.
It gives them power
so they fall asleep.
Electric.
- They are electrocuted?
Then they die.
- Kill them?
Daddy, no!
- It is not.
They are all okay, I promise.
Look at this.
He still sleeps.
It's dead, Dad.
A porch swing.
I know what I'm doing this weekend.
Now he is dead.
Welcome back to 1978.
The summerhouse.
- Not just the same.
Was it really that big?
This is my kitchen.
- Enough about that.
Thank you so much.
Can you take me a favor.
- Is that an 'Outward Bound' thing?
It looks like an episode of Lost.
That would not you rather save.
- I play it like that.
Who gets which room?
I have mapped.
I reserved the water bath for
Gloria and myself.
You're doing the backstroke last night.
I want to swim with you tonight.
- We do not really swim ...
I mean ...
- That they should not know.
I do not even know what you mean.
- I do ...
Back to the room allocation.
Rob, do you want us around?
Yes, I do.
I want one near the toilet.
- Why can not the grandmother or swim?
Here is the Big Daddy.
The master bedroom.
Lenny must take.
No, let the children to him.
It's great for the kids,
to play together.
Come, share the room.
Do not you think?
Dad, what's the big box behind the TV?
The rest of the TV.
They did not always flat screens.
This is stuff from prehistoric times.
Yes, it is.
- I got a room with heat needed.
It's 95 degrees here or here.
- And it begins again.
I have bunions.
I have a remedy, with choirs ...
- Corn is corn, say.
Yes, he wants to sound mysterious.
- I like it so well.
Choirs.
It's fantastic.
- It's amazing.
Grease it and heats it.
Go straight to the infection.
My infection is damn big.
I did not know the olifantenman
lived in your shoe.
How can anyone here ever sleep
with Tobey O'Brien under the same roof.
You step on the border, woman.
What happened to the labradoodle?
Sounds like Steven Hawking.
His vocal cords are cut.
The neighbors complained, so the
was either that or kill him.
Wrong choice.
- He is part of the family.
Like a burglar hears
He certainly walks away.
"How much the turkey in the house?".
Besides, what happened to your son?
What about breastfeeding?
His beard will soon
fused with the breast.
He gets a milk mustache over
his real mustache.
When he was ten years and cycling
sits in the basket and his ma ...
Wait, I can.
Plus 50 points.
- Why are they in?
Why not enjoy the lake?
Or something in the woods doing?
Where was the time I was not sitting outside?
That time was not there.
We were always outside.
There was blood everywhere.
- I can not hear it.
Would he do it?
We had a high score.
- No more games.
No more messages and cell phones.
Keep playing outside now.
What should we do?
- Play as normal children.
Where's Becky?
Greg, when we were children convinced
our crazy things to your father.
That's right.
- I remember ...
Our shower curtains in one basket,
and went "sailing basket.
What sent you?
- Not.
And braking?
- By aanbotsen something.
We shot caps at each other.
Always someone to dazzle.
That is too much.
- No, do not ...
not above the face.
Find a tree.
We start.
Rob has one year as a long lived.
- Why?
My father took from me my skates.
- Flashback.
Wait, you mean this?
- What?
Nice.
- Rope to a tree.
You know what that means.
- We hang?
Seriously? You see a rope and
one more and you do not go crazy?
I would by ...
You know what ...
Let me.
Climb the highest rock.
And backward.
Forward.
Who wants a double splons see?
Now let go.
- This was n mistake.
I can not let go.
Too scared.
That was great!
God, my leg is broken.
The bone sticks out.
I need help, it's ...
a branch.
Tricked.
Very good.
- This bird is hurt.
I get a shoe box.
You really landed on the bird?
- Not sure, I heard chirping ...
and crush a composite sound.
This can be anything.
Remember that we came here
after school when we were drunk.
What is drunk?
That is something that happens ...
If you crave ice.
- I do not want to be drunk.
I every day of my life.
- I chocolate drunk.
That should not.
Let me make this simple ...
That's for me.
What do you get?
This is for everyone and put it on the account.
- It should not.
I do not hamburgers.
I take minced ...
what Calzones,
the sauce separately ...
With corn or whatever.
Go in the electric chair?
He takes Henry the 8th.
She eats two.
She and Tobey Cop.
They have rings knot.
A good, Lenny.
- And to drink?
I pay, so easy.
Just kidding.
You can drink whatever you want.
I start with a Coke.
Deanne and wants a Sprite.
And ...
He has enough milk.
- I want cookies.
Cookies and MILF.
- Milf? Thank you.
Calm down.
I do flat water, Voss.
- What?
Voss and Fiji if you do not have Voss.
Whatever you've been good.
- The Voss' are sold out.
And a Hose?
- Tap water.
What country is this?
- Remove the water now but.
We do not talk about water.
Tap water.
I can take a Voss.
- View the menu but. The test.
What now?
- He says they have the best people here.
Really? Where did you learn Chinese?
I learned when I went to
The Chinese looked showcooking.
Maybe you should focus less
on talking and more cooking.
You let your away.
You have a thick toe.
- He takes her back.
Mom, can I still change her milk?
There he is. Joe Hollywood.
- Dickie Bailey?
Long time no see.
How are you?
That game had some bad things.
- What game?
That game when we were kids?
That last shot. Your foot was on the line.
Could not be counted.
I remember not really.
There's the rest of those provisions to five.
Yes, the boys are here.
- The boys?
She still does not get at her toilet.
Are you a few kilograms here?
- Ever since I was 12?
Yes, I think so.
You do not know how to deal with me.
New game, I dare you.
In basketball?
You're not in shape.
I want a bowl game off the couch '
with you, but ...
Whenever you want.
- Just kidding. I do not want to do.
If you get into the city,
I Boss that water for your child.
I do not want him thirsty.
It is also Voss water with a 'V'.
Never mind.
It is an emotional day tomorrow.
The Buzzer scatter his ashes.
The coach asked that his remains would we
scattering on Mclntyre Island.
I have a beautiful ceremony planned.
- Can you do tomorrow?
What time are you leaving?
- What are you talking about?
Are you going away?
- I had forgotten to tell you ...
we should go away tomorrow.
- You can not. Tomorrow is the ceremony.
Lenny takes everyone in the grind.
- Come on.
Are you kidding?
- I would have to be ...
but we need to Milan.
- Milan? Italy?
What's in Milan?
- The autumn collection is from Roxanne ...
Fashion Week.
- Three nights at the Four Seasons ...
eggs benedict at the pool.
Playstation 3 in each room.
And they show breasts on Italian TV.
I'm going to Italy.
We are all going to Italy.
I was just surprised.
Lenny, it's not the same without you.
Roxanne allowed us to tonight,
This means a lot to her.
Roxanne can not go without you?
We are a team, my lady and me.
We do everything together.
Last year we had Becky's play
skip to Milan.
I thought we were drunk are.
What should I do, honey?
Lying to them?
It's not what you said, but
what you said.
I'm not an insensitive
loves her work dragon woman.
And I will not let you
pretend I am.
We turn it over dinner in Milan,
scatter the ashes ...
and we will leave early Sunday morning.
Lamonsoff, it's five hours in the morning,
You can kill the dog?
I think about it.
Forward. Here we come.
- Good morning.
On the table.
What stinks here so?
Gloria and I make breakfast for everyone.
We use a dehydrator.
It cooks anything above 150 degrees.
Are those pancakes?
- Those are pancakes Ocean.
They look great. Like them or
of one leg of an elephant have cut.
That is good for you.
- What is this?
Is this a failure?
- A dried banana.
And then an ordinary banana?
Peeling and eating.
The kids will love.
It is not bad.
You cut it into pieces ...
- I see what you do ...
Maybe you're right.
- That you can do.
I prefer to be beaten with dried fruit
then edited.
Processed fruits.
- I'm just kidding.
You think because you're rich, you people
can store dried fruit.
I think not.
Do not say that.
Good morning.
How's the bird?
Those shoes are so ...
- I'm too neatly dressed, I know.
I had packed for Milan
not for the lake.
Made from acorns.
- I do not eat ...
you still eat normally. Cornflakes?
- Will I make Frittata?
Do you think now in wetenschapsles?
Frittata make?
You can start practicing in your room.
That big test is coming.
I think you Lenny overwhelms.
I have a gift for you.
Bacon. Go ahead.
- Bacon.
She gets it.
- She gets it.
She gets it.
- Thank God, she gets it.
What do you do?
- The bacon.
Not here. That meat comes from dead animals
away from the vegetarian food.
Killer.
- Good.
What do I do with it?
I know. Sorry.
Curley, shame on you.
That crazy dog to my leftovers
pasta have sat.
Yes, mommy, that's certainly happened.
- Really.
We would have to take a dog.
- We would have to take a water buffalo.
Higgins, what are you doing?
I wear no pants in my house,
consequently, in your house.
Was that a man's ass?
Look at that.
Well done, lad.
Throw the stone.
Throw it into a tree.
Throw it in the lake.
Throw it at your brother.
Touch your head with it, but do something.
Gregory, why are you dressed like that?
Do you deal with Jiminy Cricket?
We would go to Milan, remember?
That I have packed.
Put that in your pants then we play 'hole player.
- I pass.
Do not you want revenge for that last time
I beat you?
That was four years ago.
I just want to fly to Milan
Enchilado and fresh food.
What would your father have done it?
That would be a blow against the side of
head have been ...
possibly with blood from his ear.
My father would have thrown me down the stairs
alone to speak Enchilado.
You have new Formet.
- Yeah, great.
He wanted to make Tofu grape-nuts.
- Try it out.
You did clean?
- Yes.
Thanks.
- Mclntyre Island is not closed.
There is a big role for us.
You have not cleaned it, huh?
Come on, man.
What are you doing to me.
Your breath does not smell it
like your grandmother's closet.
Respect but we go
Mclntyre Island go ...
I have not done
since we were 12.
There I got my first boob.
- I know. Tracy Tornhill.
They were or flat, but
I was a fan.
Higgins, can you just think about girls?
We have daughters.
- You do not think that just on.
but I can do something about it.
- That's not true. Well, that's true.
Want to know where I fantasize about?
- Yes.
Bars.
- Bars?
My last blood test was not good,
and I got nothing of Sally.
You're a grown man.
I have my own cache.
I'm not stupid.
Everything, Butterfingers, 3 Musketeers ...
which I hide under my clothes,
at the bottom of the hamper.
Because the smell of my sweat socks
hide the chocolate scent.
And as your underwear looks like,
that hides the chocolate stains.
I gotta pee.
You've already done three times.
You're leaking.
Shut up.
I try to concentrate.
You just pee, no test.
Are you urinate, or turn
the diesel off. What is that?
Listen to him.
He sends a message in Morse code.
Getting old, stop.
Can not urinate, stop.
Smells like asparagus, stop.
Even though I ate not.
The good news is that those girls
there can say goodbye to.
That is so awful.
- Did you see that?
This is how they do not all out, ladies.
Guys, I've also brought something.
Go away.
- The network of the Championship.
Let's see.
- I thought that the coach should have.
You must deposit the urn.
Your foot was on the line!
- Bailey ...
What would the coach said
if he had heard talk dirty?
Us a kick ass we give
because we did not do that.
Whether we do it the old fashioned way,
Eric beats him up.
No, I only have eight seconds left to
fight in my life.
I will save that for when I robbed,
or if someone calls at Burger King.
I can no longer postpone.
- He just put the chicken in his mouth.
Are we?
- We are going to do it.
I think the coach is here with us.
I think he is proud to be here
be with us.
As ash.
- Here on this island ...
where we hid our parents and
Now for our women and children.
Goodbye, friend.
Wise counselor.
The buzzing noise maker.
Go ahead.
- I've ruined my marriage.
They were all my fault.
I could not keep up.
I barely know my own daughters.
Buzzard would be ashamed of.
- False.
You have enough time to explain to
with your daughters.
Call them when you return.
Actually, they have about 20 minutes.
What?
- I can use a piece of chicken.
No. The hands of the coach.
- Make the shell but empty.
I'm going to eat it anyway.
Sick, but I'm hungry.
Stupid squad car.
Hi, Dad.
- I?
How did you do that, that get?
And I am not exaggerating.
How is that possible?
Elvis goes to her in her nose.
She does not know what to do.
Is this your car?
My old man lent me this rotauto.
It makes him not look me in limbo
beach and get eaten by a bear.
Your old man?
- Yes, my friend.
of boys of my age did.
But I'm the father chasing
who let me in the lurch.
Wu Ze-mama.
- What does this mean?
No idea, but it feels good.
I hope this car never made.
I do not think so.
I'm the spark plug removed.
We look all at once.
Let us each in turn.
Yes, I will look at the tree.
- McKenzie, you first.
Good plan.
And exercise.
And exercise.
What do we do?
- If we look at all the tree?
Back to the ass.
This is Gloria, this is Jasmine.
Grandma Gloria comes to the rescue deal.
Let me see.
Honey, your gasket is blown.
You had to see them in the Carwash.
You had to hear that waterbed action.
Just two Grizzly's fighting
a salmon.
He gets something.
I left out since she was pregnant.
- That can happen.
Can you have sex when they are pregnant,
or damage the baby then?
McKenzie can do that because the baby
thinks that his feet are tickled.
I was lucky on St. Patrick's Day.
She was half asleep.
With half asleep, I mean
they slept.
That you can do.
- She agreed.
What happened with your daughter?
- Embarrassing.
The first 20 years of a child's life
are more important than I realized.
It's always the children who pay.
- Children pay.
They always pay.
They pay with grain.
Good talking with you, boys ...
- No, no.
My family has become entertainment
for you.
How we brightened Robbie earlier?
Bow and arrow contest?
Have we really done?
- The second dumbest we ever did.
It was the stupidest.
- Dementia.
Arcenicum.
I can not believe we do.
How do we always
talk round by Lenny?
Remember the rules, who
remains the longest, wins.
Anyone who stays longest.
One and two ...
- Why are we doing this?
I lost him in the Sun
I am the champion.
I'm playing you queers.
There is an arrow in my foot.
Calm down.
What's going on?
- Rob was attacked by Indians.
But that has escaped.
- Attacked by an Indian?
Daddy, what happened?
- Amber, I'm glad you're here.
This is my daughter Amber.
Another one?
- High Five.
Now look.
- I'm dying.
Bridget, come here, daddy is hurt.
- Do you have one?
That is logical.
- Shut up.
Pulls him out.
- I think we have to push him through.
My idea, I said it first!
It'll be fine.
We just need some cream corn ...
- Shut up.
That shit is not working.
That never worked.
Are you insane?
Add some alcohol.
As you know enough.
You were born with a prohibition.
And that is pretty.
- Bigger than I thought.
Immense.
Abnormal pancreas form ...
Sorry, Princess Rita, what you're sitting there
or do you help me?
Mr. Feder said I had to learn
for the big exam.
Which exam?
- No idea, he keeps giving me books.
Have you caught the adapters for Italy?
- Yes, in L.A.
Wipe your hands, you have all the papers
neat storage ...
they should not be damaged.
Feder with Roxanne, I want to be sure ...
- Mum, it's true.
I'm on the phone.
- My tooth has fallen out.
Put it under the pillow,
then I explain it later to one U.S. dollars.
What?
- I'm on the phone ...
I stop it later one U.S. dollar ...
Why would you do?
There is no tooth fairy?
I can not believe I
just did.
You should have seen her face.
What's wrong with me?
So I do my best.
And I can not do well.
- Parents make mistakes.
Remember when I told Greg that
babies of mothers were poo?
And he kept looking in the toilet
a baby brother?
That's not funny.
- I'm just saying.
I feel like her childhood
I screwed up.
She comes back eventually anyway.
- Eventually, but not today.
I did not say 'no mobiles'?
I thought we had agreed on something?
What is it, Daddy?
- Are those cup telephones?
Yes, Charlie has taught us that. Cool, huh?
Look at that Becky.
Look, Dad, there is a waiting cup.
I'll call you back, call me Sponge Bob.
Was that a turkey?
- Andre, can you hear me?
Mr. Feder, you hear me?
The best part is that we
a cup in the shower let ...
Jasmine and the clean up tomorrow morning.
- What?
Attention.
I have a question from the telephone company.
If you would like, in every adult bedroom,
cup phones installed?
I've always been the tooth fairy.
No, Bean's mom is the tooth fairy
for his family.
I do it just for our family.
No, do not tell Bean.
Thank you.
I love you so much.
I'm sorry, honey.
- This cost me a fortune.
Hang. This is a high
'Cup account.
It is nice to your feet to make cereal.
Is that Gloria who laughs? Is she still
here after he snubbed her so it?
She forgave him immediately.
You're my special lady.
- And you my eenvoetige grain-covered man.
We have to go to sleep.
Tomorrow we go to Milan.
That's a long flight.
We are going to rest.
Take off that turkey!
Word I get horny from?
Yes.
Promise you will still feed
after we have gone away?
I promise.
- Mama, that's my milk.
We must learn to share.
Even with birds.
But He'll drink up.
Want to try?
- Yes.
But I can not exercise.
- That's okay.
Put it this way, a bit down,
throw the stone and do the rest.
Do you know how to skip stones?
I'm the best.
Note.
Is everything okay?
- I am glad to see that you have fun.
I bring your luggage to the car, Mr. Feder.
That is not my luggage.
Glad your own stuff
bring down.
Succeed?
- Going?
I gotta go.
- Want me to the airport?
We have a small problem.
- What happened?
I forgot how to jump a stone fell.
I think that takes a few days
before I'm out.
We had better stay.
Are you kidding?
- Are we really not going away?
I do not starving models
in my clothes walking around to see.
Why would you want to see models,
if you can see.
See? You do not get that in Milan.
What do you do today?
Say it.
No idea, but you say it.
What did you love when you were 12,
and the sun shone.
I can not.
Perhaps to a water park.
- What are we waiting for?
Really?
- Let the car load.
I go get my swimsuit.
You are wasting it.
- Sorry.
Okay.
It tastes so bad yet.
We can sit here.
I have to do two nummertje.
Hi, Dad.
- Where were you?
We forgot our bikinis.
So Mark bought us.
Bought in a shop called
"Naughty At Night". The only one who was open.
There was only one thing open?
For bathing suits?
In the summer?
- My swimsuit is from the Seventh Shop.
I like that.
- On the water slide.
What a beautiful day.
It is a meisjesdag.
- It's hot.
Are we really in a swimsuit?
We know that people do not.
What do I even care. I have
all summer working on this six pack.
How about that?
A large front.
Come on, The.
Your turn.
Pull out, baby.
No, I'm cold.
And you, Gloria?
I have no bathing suit.
But I like to walk naked.
No.
Someone likes what he sees.
The row is not even ahead.
I want mommy's milk.
I know ...
- Give his mom's milk.
No.
- Mama's milk.
No, damn it.
No more mama's milk.
If you want milk, drink your milk but ordinary.
Enter here. Here's a dollar.
Walk on.
But drinking milk. Straight from the packet.
- I do not know if Mom wants.
Drink it now.
Just like Dad. I show you.
I'm just like daddy.
See you.
No 'breast milk' more.
More for us.
How do we ask Mr. Banana Hammock to here,
so we can see it better?
There are two major advantages of breastfeeding,
and I put mine to work.
Well done, girl.
Dance but that pole, girl.
This works for sure.
- There you are good at.
He likes it.
So we avoid the row.
Can we get through?
- No way, but we do it anyway.
Cool dad is back.
The ladder.
It's dark inside,
maybe you should hold my hand.
Not on him fall.
Stay away from him.
What are you talking about?
He's a gay?
No, it looks only.
He is a genius.
Next.
Now is the right time.
You first.
Is he this way?
- He comes to us now.
What you do,
let no winds.
How is it, ladies?
Are you here?
I'm from Saskatoon.
That's in Canada.
You American ladies,
have come to Saskatoon.
Bullies.
Everything was filled with steroid
except his voice.
Next.
Who's next?
Were you just got here too?
No, you're mistaken with someone else.
I do not think so. You are the two
horniest girls I have ever seen.
What?
- These are my daughters.
Yes, yes.
She sure.
It is ugly, just like you.
Who's next?
Hurry up, I want to go diving board.
Wait, we do so immediately.
Are you urinating?
Not true.
They make chemicals in the pool
and thereby urine blue.
No, that's just talk.
It shames me.
Mama, we take a picture?
Well, a little more back.
More.
Gotcha.
I am still not sure.
It's pretty style.
He is steeper than I remember.
- I think it is anything but.
Come on, we go away.
Well the child was frightened,
then you do not have to go.
Why do not you hire someone who
for you, rich guy?
Wiley, how are you?
Go once to the side,
can I attend.
Goodbye, lafaardjes.
Very nice.
- Now I want to.
See that?
Backwards?
You're next, Hollywood.
- I?
I do not know what you're after back and forth,
but watch the descent.
Again the second place, Bailey.
You hear him?
Second place.
Second place.
Get on your feet hanging.
Let those idiots surprised.
- Getting my feet?
Yes.
- Is that safe?
You're a beast.
I am a beast.
Waardom you doing it?
- I have kids.
You do not have to do.
I am a beast.
Well done.
He is apparently completely forgotten me.
Typical men.
They take all you can give and ...
then they have you in exchange for a cow.
I do not know what to do with this now.
- I do.
Did you see that?
That is the "bank shot".
Choose a spot.
Your father is 50 for 50 with this.
Really? I think is a "bank shot" never.
You must return,
and your money refunded.
Ted Dunkin won there
If you look at me?
- That was great.
Come here I'll let you see.
Nice show.
You see the basket?
The upper right corner?
Concentrate.
Lamonsoff, focus.
My God, what a pervert.
He looks at me completely.
- We looked not to ...
Come on, come on.
Hands up.
You're amazing.
See how good you are,
if your normal clothes?
Why are you not to play basketball?
- You know I ...
You have calluses yes.
Where's Kurt?
Not afraid.
Someone has issues.
That does not look good.
I've done your first shot,
Now you do the second.
Nobody is doing shots.
There are children here.
I have to wake up tomorrow morning.
- Then we do beer.
Who needs beer?
- I got one.
I stick to coffee.
- I got this.
I only alcohol tonight.
What about you?
We came here all parties.
You are sometimes grown.
You just meet the right girl.
My older sister would be perfect for you.
She is officially blind, but they can
shapes.
Roxy, how have you and Lenny
know each other?
Good question. How did this piece of ignorance,
meant to be offensive ...
met this beautiful woman.
He came to the store where I worked.
I helped him.
He wanted me to fit a skirt.
A skirt fit. Good trick.
Very sneaky.
- He let me catch up ...
Then I had to pick a bra.
I just wanted to help.
I thought he was a little fun.
It was so good.
- My water broke.
The baby comes.
I had you and you and your address.
I'm just kidding.
Do you know this song?
That's great.
This song is great.
I'm so horny for this song.
I thought, but not anymore.
I want to be credited for it.
This is all part of my
"Is so happy" playlist.
Ladies on the dance floor.
I have my lady here.
I know how happy you are dancing.
- I am a true "Shabanu."
Look at you again. So sexy.
Change your name Deyonce.
- Now you find me sexy, huh?
Can I enjoy a dance
with my beautiful wife?
Yes, the best song.
- The best song ever.
You were right for us
you go to take.
It is very special.
- Right?
There is nothing going on
and I like that.
I am talking with 25 different girls
done, and you should be number 26.
If I am but the last.
- Guaranteed.
I am horny.
- What are you doing here.
Lamonsoff, what are you doing awake?
Today was crazy.
Sally struck me as never before.
It was crazy.
Like with my wife.
- Really?
Where is Saskatoon?
Yes, I do not know.
Caesar!
Look.
- What are you?
Caesar.
- Where are you from?
Saskatoon.
What about Higgins?
I am envious of here, you may
not so drunk to bottom.
Unmarried, without liability.
Without being hurt.
- He must be hurt.
Honey, you're back on the couch
asleep.
You know you go to your room
wanted to go.
The bird is all better, Mark.
He wants to talk.
Mark, you are the best lifeguard
the entire pool.
Hold your breath, honey.
Who is that?
- Mama's boyfriend ...
who dumps you.
He cries.
It is good.
- We bring you up, dear.
I put on your pajamas.
- You can not just sleep anywhere.
Go to your room.
This is not a cabinet, but your bedroom.
It's your toilet, you can pee.
I pee.
Happy July 4th, everyone.
For the bird was also Independence Day.
So thumbs, baby.
Bye, birdie.
Not so sexy. What did she say?
- I think she said ...
Those guys in the shop were so dirty
talking about that basketball game.
Not until the uniforms are ready.
No more drinking.
Did you really sleep in the closet, drunk?
I think so.
No ...
I had the hottest dream
a little blonde girl.
I think it was a dream.
These eggs will be good if you
Marinara sauce is what it does.
Lenny his phone.
Hello?
Here we go.
There are three. Ladder.
Tell him what is going on.
He may not care.
- Tell him.
Gloria and I think of a child.
- How great is that?
That's great. Good news,
more stretch marks can not.
A good one.
That Gloria and the children can share diapers.
That's just bad.
- He's not in 'n committed relationship ...
He got it not because he
No relationship can hold.
He's just mad because all my girls
every 70-80 years younger.
My last one was blond, big
and super hot, not your type ...
and we go for it.
You go for it.
No idea what that means.
And the good part,
there is a picture of the wall ...
right over her head.
- She cuts her head?
The cut hair and I delayed not.
I threw two aspirins
and she snaps there ...
I'm tired, Marcus.
Your snoring kept me awake.
Leave me alone, crazy.
You're married.
This time you go pay.
Turtle.
- You pig.
Robby, let go.
Take it easy.
Calm down, buddy.
What is the problem?
The arrow foot.
Why do you fight?
- Still not wrestle again?
How could you make?
That's my daughter.
What to do?
- The bed sharing with her.
I'd call that, do you think?
- From us, would you are.
Stimpson you slept with his mother.
- I hardly knew him, that friend of yours.
She clutched my frankfurter.
I told you.
That is his daughter, that's different.
- He and I?
Maybe I flirted.
But just for attention.
What about his snoring?
- I heard that too.
No! Twice?
- Even with three rooms away.
You know Mark your daughter would not ...
Not that she ...
You understand me.
What happened to your head?
- When I picked my pajamas ...
seemed to me a pervert
looked through the window ...
So I wanted to close the curtains
and stumbled.
Dude, it's your daughter.
I would never do.
Jasmine and not with me
because I am.
I have my life like Lenny.
Not like Lenny.
Please. I took your phone.
It was your assistant.
I asked if his flight to
Milan had been canceled ...
and he did before we
came here!
He said that?
Why would he do that?
So he invented that?
- I did not say ...
In his mind he believes that.
Mothered me, Lenny.
Do not check everyone and everything.
Or to find a time
have some responsibility.
I will do that.
You're right.
I thought it was better for our family
here are normal people than in Milan.
Are we normal?
- It was easier for the secret to keep.
In every marriage there are secrets.
Mine is that I very much
love you.
Roxanne, our children are so spoiled.
We have a nanny ...
- So that they do.
No, she is not ...
- Of course she is a nanny.
When does the line?
- After.
It is more an au pair
than a nanny.
Well, the line stops here.
I will never lie to you.
I swear.
Men lie.
They can not be better.
Kurt lying so much.
- What, O.J. Simpson?
Please, the nanny.
- What is it?
You feel something for her.
- That's not true.
I am married. Forever.
But if I enjoyed a
great interview ...
a woman who asked:
Where did this or did you get this picked up?
Yes.
- You can with me.
Where? It always works.
When you get home you're always tired.
You barely touch me or look at me.
He heard an old woman.
Deanne, have you had with Umberto Denusio
married as I told you.
And if I had stayed with him,
You were now pregnant.
You probably right, baby.
I remain silent now.
Thank you.
Maybe I do not recognize
and I'm sorry.
What if I do every Thursday
your expenditure?
Thursday is Grey's Anatomy
but every other day going.
Since we are all honest,
I must be too.
I'm not a co-owner
of lawn furniture.
I even work there anymore.
Layoffs in April.
And I rented the convertible ...
and behaved as a big boss,
You did not think so ...
You should not hide from us.
Sorry.
Now we express it here ...
I must confess something.
I wear a toupee.
- You mean it.
I also want to confess something.
I'm actually a man.
I told you.
No, I laugh there with me.
I'm pretty offended that
believed that much.
No, we said something.
- Anyway ...
I know it's not my place ...
but I'm still older than everyone.
- That will be hell!
Despite the jokes and laughter
I see a lot of love here.
And with love comes with hostility.
As Rob told me that erupted.
That was scary.
And inside I said ...
Gloria, do not call him sociopath
with some male complex.
Do not say that he did through his hair
looks like a dirty cotton swab.
Or that he's one of the gay
the Jonas Brothers proposes.
They understand what you mean.
Or a dwarf Filipino Fonzie.
I said nothing.
That's my favorite.
Life is difficult sometimes.
Bumpy.
But with family and children
and unscheduled things ...
but just make it interesting.
During life, the first act
always exciting.
The second act ...
here comes the worst.
Why you told us that she never gets it.
I tried that too.
Group hug!
Hold your position.
- It looks odd.
It's okay.
I have worked all morning.
It would be good to come.
- Should?
Jump.
Again!
It is not grace,
but a good investment for me.
A car that is not there.
It can be a gold mine.
I would not take your money.
- You do not.
I do too.
I know the owner of the Celtix.
I can provide that account.
That guy drives Bentleys and Ferraris.
How do you know the owner of the Celtix?
I am the leader of that
girls scouts group.
If he wants her to get that cookie badge,
he gives us that account.
Ready for some fun?
Yes, go ahead.
As hard as possible.
Would you really do?
- Five of me from your five.
The boys have stayed in shape.
Robideaux, good to see you.
- Lenny.
Are you looking at me? I do not know.
We just want to eat watermelon
and relax.
No, they do not run away.
Wiley escaped from the hospital
to be able to see your justification.
Bring the ball there but, Feder.
Was it a good field goal?
Yes, the first time I heard that.
- Okay, sorry.
"You want some of the top shelf to take?".
Which I have often heard.
Let us just with our family
hang out and relax.
Yes, they are afraid of us.
We are afraid that you eat on the ball,
before we started.
Same old Higgins,
He can not talk ...
The rest I forgot.
Rob has a bad foot.
You can not, huh?
Buzzer as here,
I wish he tried.
That we all tried.
Is that William Shatner?
Let's do this.
Game to 21.
Come on.
Guys, look!
Here is the other black
who grew up here.
No, here I am black
and you another.
No, I'm black here.
When white people see me in the shop,
they get fear.
If they see me, they walk away.
Yes, these are my whites.
- You're crazy.
Fluke.
Defense!
In your face!
Guys, movement from 1978.
Show me what you got.
WMCY.
What the hell is that?
Like Alf sex.
Because of you I wasted my beer!
Why does everything about you running?
Why are you always such a dick?
I can not anymore.
- I can not even breathe.
I feel my legs anymore.
Want faking injury?
- Definitely.
I can not go.
Baby.
- I'm exhausted.
Soon someone has a heart attack.
What are you doing?
Is that all?
- Darth Vader, sit down.
Is that an ice cream from Firgillio?
Did you seriously just now the
ice for your daughter?
From Firgillio.
Leave me alone.
Not choke, Dad.
Let your and my son
bring it?
Good.
- Greg.
Come on, baby, you can.
- Bailey.
Let someone who bottle
what Boss of water.
It's Voss with a V.
Can I touch your belly?
I feel like baby steps.
Yes, go ahead.
- Thanks.
That is not my baby
but my boob.
I feel like that.
Nothing wrong.
Well, from 1960 to 1960.
Foot of the line this time.
The cost for Hollywood.
Your foot was on the line!
That is not on.
You can not.
- Good shot.
Next basket wins.
Shoot him in it.
Throwing, Dad.
Such steps, he never misses.
There goes Hollywood.
Sorry, Dad.
We deal with them next year.
I go for training.
So let me know. Come here.
Lenny, I thought you'd never
missed a bank shot?
I thought the Bailey's must learn to win ...
Feder and to learn what losing is.
That was sweet.
I know you will rot,
you have lost the game.
And are you Sat
- No. Yes.
There is one thing they never beat us.
- What?
Archery.
We are really good.
Okay, everybody.
- What are you doing?
The last one still standing wins.
- Running away.
On the side.
Not again!
Take me with you.
Let me seriously behind?
No!
We win again!



'''

